In [3]:
import torch
import snntorch

In [16]:

output_spikes = torch.tensor([
    [0.0, 0.0, 1.0, 0.0, 0.0],
    [1.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 1.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 1.0],
    [0.0, 0.0, 0.0, 0.0, 1.0],
    [1.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0],
])


output_spikes_batch = torch.tensor([[
    [0.0, 0.0, 1.0, 0.0, 0.0],
    [1.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 1.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 1.0],
    [0.0, 0.0, 0.0, 0.0, 1.0],
    [1.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0],
    ],
    [
    [1.0, 1.0, 1.0, 0.0, 1.0],
    [1.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 1.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 1.0],
    [0.0, 1.0, 0.0, 0.0, 1.0],
    [1.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0],
]])

print(output_spikes.shape)
print(output_spikes_batch.shape)

torch.Size([8, 5])
torch.Size([2, 8, 5])


In [19]:
def decode_count(spikes):
    spike_counts = torch.sum(spikes, dim=1)
    action = torch.zeros(spikes.size(0))
    max_spike_count = torch.max(spike_counts)
    candidates = torch.where(spike_counts == max_spike_count)[0]
    if len(candidates) > 1:
        action[torch.multinomial(candidates.float(), 1)] = 1
    else:
        action[candidates] = 1
    return action

In [6]:
def decode_time_to_first_spike(spike_trains):
    
    decoded_values = torch.full((spike_trains.size(0),), spike_trains.shape[0] + 1)
    print(decoded_values)

    for i, train in enumerate(spike_trains):

        first_spike_time = torch.argmax(train) 

        if train[first_spike_time] == 1:
            decoded_values[i] = first_spike_time.float() + 1

    return decoded_values

In [7]:
def decode_first_spike(spike_trains):
    decoded_vector = [spike_trains.size(0)+1] * spike_trains.size(1)
    
    for neuron_idx in range(spike_trains.size(1)):
        first_spike = (spike_trains[:, neuron_idx] == 1).nonzero(as_tuple=True)[0]
        if first_spike.nelement() != 0:
            decoded_vector[neuron_idx] = first_spike[0].item() + 1
    
    return torch.FloatTensor(decoded_vector)

In [11]:
def decode_first_spike_batched(spike_trains):
    """
    Decodes the first spike time from spike trains for batched data using 'time to first spike' method.

    Parameters:
        spike_trains - The batched spike trains with shape (batch_size, num_steps, num_neurons).

    Returns:
        decoded_vector - The decoded first spike times with shape (batch_size, num_neurons).
    """
    batch_size = spike_trains.size(0)
    num_neurons = spike_trains.size(2)
    decoded_vectors = []

    for batch_idx in range(batch_size):
        decoded_vector = [spike_trains.size(1)+1] * num_neurons
        
        for neuron_idx in range(num_neurons):
            first_spike = (spike_trains[batch_idx, :, neuron_idx] == 1).nonzero(as_tuple=True)[0]
            if first_spike.nelement() != 0:
                decoded_vector[neuron_idx] = first_spike[0].item() + 1
        
        decoded_vectors.append(decoded_vector)

    return torch.FloatTensor(decoded_vectors)

In [18]:
decoded_trains_batch = decode_first_spike_batched(output_spikes_batch)

print(decoded_trains_batch)

tensor([[2., 3., 1., 9., 5.],
        [1., 1., 1., 9., 1.]])


In [22]:
# Decode the output spikes
decoded_trains_batch = decode_count(output_spikes)
print(decoded_trains_batch)

tensor([0., 0., 0., 1., 0., 0., 0., 0.])


In [43]:


p_actions = torch.nn.functional.softmax(-decoded_trains, dim=0)
print(p_actions)

tensor([2.4173e-01, 8.8927e-02, 6.5709e-01, 2.2043e-04, 1.2035e-02])


In [141]:
import torch

lidar = torch.tensor([0, 0, 0, 300, 300, 300, 300, 300, 300])  # Your LIDAR/vision rays data as a PyTorch tensor
halfwinsize = 5  # Example window size

max_input = 300

torch.manual_seed(2)
wraparound_data = torch.cat([lidar[-halfwinsize:], lidar, lidar[:halfwinsize]]).float()

print(wraparound_data)

conv = torch.nn.Conv1d(in_channels=1, out_channels=1, kernel_size=(2*halfwinsize+1), padding='valid')

tensor([300., 300., 300., 300., 300.,   0.,   0.,   0., 300., 300., 300., 300.,
        300., 300.,   0.,   0.,   0., 300., 300.])


In [145]:
maximum = (conv.weight.data.clamp(min=0).sum() * max_input+ conv.bias.data).item()
minimum = (conv.weight.data.clamp(max=0).sum() * max_input + conv.bias.data).item()

138.28907775878906


In [154]:
def normalize_vec(vec, maximum=None, minimum=None):

    if maximum is None or minimum is None:
        assert maximum is None and minimum is None
        maximum = max(vec)
        minimum = min(vec)


    max_val = max(abs(maximum), abs(minimum))
    maximum = max_val
    minimum = -max_val 

    return [2 * ((x-minimum)/(maximum-minimum)) - 1 for x in vec]

In [155]:
convolved_data = conv(wraparound_data.unsqueeze(0).unsqueeze(0))

print(normalize_vec(convolved_data.squeeze().squeeze().detach().numpy(), maximum=maximum, minimum=minimum))

[0.05548017220892354, -0.5336056710803933, -0.5640488840760428, -0.5812859011533809, -0.32683276079140666, -0.038462985295128815, -0.016573731645048384, 0.31805901652000945, 0.060132418834491386]
